In [2]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

digits_df = pd.read_csv("train.csv")
digits_df.shape

(42000, 785)

In [3]:
x = digits_df.values[:, 1:]
y = digits_df.values[:, 0]

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784,500)
        self.fc2 = nn.Linear(500,250)
        self.fc3 = nn.Linear(250,125)
        self.fc4 = nn.Linear(125,10)
        self.bn1 = nn.BatchNorm1d(500)
        self.bn2 = nn.BatchNorm1d(250)
        self.bn3 = nn.BatchNorm1d(125)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.relu(self.bn3(self.fc3(x)))
        x = self.fc4(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
#Training and testing data, converted to torch
train_size = 0
data_test = torch.from_numpy(x[:train_size,:]).float()
target_test = torch.from_numpy(y[:train_size]).long()
data = torch.from_numpy(x[train_size:,:]).float()
target = torch.from_numpy(y[train_size:]).long()

#Setup model
batch_size = 64
model = Net()
optimizer = optim.Adadelta(model.parameters(), lr=0.1)  #Learning rate

#Training
for i in range(30):
    #Training mode, run data through neural network
    model.train()
    #Loop over minibatches
    for j in range(0,42000-train_size,batch_size):
        data_minibatch = data[j:j+batch_size,:]
        target_minibatch = target[j:j+batch_size]
        optimizer.zero_grad()
        loss_SGD = F.nll_loss(model(data_minibatch), target_minibatch)
        loss_SGD.backward()
        optimizer.step()

    #Accuracy
    model.eval()
    with torch.no_grad():
        test_pred = torch.argmax(model(data_test),axis=1)
        test_accuracy = torch.mean((test_pred == target_test).float())
        train_pred = torch.argmax(model(data),axis=1)
        train_accuracy = torch.mean((train_pred == target).float())
        print('Iter:%d, Test Accuracy=%.2f, Training Accuracy=%.2f'%(i,test_accuracy*100,train_accuracy*100))

Iter:0, Test Accuracy=nan, Training Accuracy=97.83
Iter:1, Test Accuracy=nan, Training Accuracy=98.92
Iter:2, Test Accuracy=nan, Training Accuracy=99.40
Iter:3, Test Accuracy=nan, Training Accuracy=99.76
Iter:4, Test Accuracy=nan, Training Accuracy=99.92
Iter:5, Test Accuracy=nan, Training Accuracy=99.96
Iter:6, Test Accuracy=nan, Training Accuracy=99.97
Iter:7, Test Accuracy=nan, Training Accuracy=99.98
Iter:8, Test Accuracy=nan, Training Accuracy=99.99
Iter:9, Test Accuracy=nan, Training Accuracy=99.99
Iter:10, Test Accuracy=nan, Training Accuracy=99.99
Iter:11, Test Accuracy=nan, Training Accuracy=99.99
Iter:12, Test Accuracy=nan, Training Accuracy=99.99
Iter:13, Test Accuracy=nan, Training Accuracy=99.99
Iter:14, Test Accuracy=nan, Training Accuracy=99.99
Iter:15, Test Accuracy=nan, Training Accuracy=99.99
Iter:16, Test Accuracy=nan, Training Accuracy=99.99
Iter:17, Test Accuracy=nan, Training Accuracy=99.99
Iter:18, Test Accuracy=nan, Training Accuracy=99.99
Iter:19, Test Accuracy

In [6]:
torch.save(model.state_dict(), "DigitRecognizer.pt")

In [7]:
test_df = pd.read_csv("test.csv")

In [8]:
ans = model(torch.from_numpy(test_df.values).float())

In [9]:
p = torch.argmax(ans, axis=1)

In [10]:
n = p.numpy()

In [11]:
list_entries = []
for i in range(len(n)):
  entry = [i+1, n[i]]
  list_entries.append(entry)

df = pd.DataFrame(list_entries, columns=["ImageId", "Label"])
df = df.set_index("ImageId")

In [13]:
te = pd.read_csv("sub.csv")
j = 0
for i in range(len(te.Label)):
  if(te.Label[i] == n[i]):
    j += 1

print(j/len(te.Label))

0.9860714285714286


In [ ]:
df.to_csv("sub.csv")

In [ ]:
df.head()